In [4]:
consumer_key=''
consumer_secret=''
access_token_key=''
access_token_secret=''
binance_private_key=""

In [5]:
import threading
import pandas as pd
from datetime import datetime
from decimal import *
import twitter
from binance.client import Client
global client
client = Client("V40EnBQtXZZLdxv7r7DmVqfJ6NFGdHn1Wp8OhTr9Nq5BnpQNl1qqdRZORGRsaYTu", binance_private_key)

def balanceOf(sym):
    info = client.get_account()
    balances = info['balances']
    for coin in balances:
        if coin['asset'] == sym:
            total = Decimal(coin['free']) + Decimal(coin['locked'])
            print(coin)
            return total
    return(-1)
    
# Settings
amt = 10 # Trading Amount in USD
symbols = []
prices = client.get_all_tickers()
for i in range(100):
    if prices[i]['symbol'][-3:] == 'BTC':
        symbols.append(prices[i]['symbol'])         # I gave it btc pairs for testing

# Global Variables
isRunning = True
buys = {}
sells = {}
isOrderOpen = {}
uptrend = {}
lastPrice = {}
quant = {}

#Symbol Specific Variables
for symbol in symbols:
    if float(balanceOf(symbol)) > 0:
        isOrderOpen[symbol] = True  # Might cause errors, need to add a more reliable check for open positions
    else:
        isOrderOpen[symbol] = False
    uptrend[symbol] = -1
    lastPrice[symbol] = 0
    quant[symbol] = {'init': amt, 'usd': amt, 'qa':0}
    buys[symbol] = []
    sells[symbol] = []
    
# Functions
def tweet(x):
    api = twitter.Api(consumer_key,consumer_secret,access_token_key,access_token_secret)
    x = x +' -Bot '+str(datetime.now())
    print(x)
    try:
        status = api.PostUpdate(x)
    except Exception as err:
        print(err.message) 
        return -1
    return 0

def srcAvg(x):
    avg = 0.0
    for i in range(1,5):     # len(x) -> 9
        avg += float(x[i])
    return avg / 4           # return average of open, high, low, close

def s(x, r = 3): # str(round())
    return str(round(x,r))

def ma(isExp, length, klines, closesOnly = 1):
    # klines = client.get_historical_klines("ETHUSDT", Client.KLINE_INTERVAL_1MINUTE, "1 hour ago UTC")
    if closesOnly:
        closes = pd.Series(data = [x[4] for x in klines])
    else:
        closes = pd.Series(data = [srcAvg(x) for x in klines])
    if isExp:
        return list(closes.ewm(span=length).mean())[-1]
    else:
        return list(closes.rolling(window=length).mean())[-1]

def getStepSize(symbol):                    # order sizes need to be in multiple of the step size for the symbol
    exchange_info = client.get_exchange_info()
    return float([x for x in exchange_info['symbols'] if x['symbol'] == symbol][0]['filters'][1]['stepSize'])

def looper():
    global symbols, isRunning, symbols
    if len(symbols) == 0:
        isRunning = 0
    for symbol in symbols:
        checkPrices(symbol)
    if isRunning:
        threading.Timer(1.0, looper).start()

def checkPrices(symbol):
    global uptrend, lastPrice, isOrderOpen, quant, symbols
    if quant[symbol]['usd'] < quant[symbol]['init'] * 9/10:
        symbols.remove(symbol)
        print(symbol + ' removed from symbols') # Remove symbol from pairs if losses exceed 10%
        
    prices = client.get_all_tickers()
    curPrice = float([x['price'] for x in prices if x['symbol'] == symbol][0])
    if abs(lastPrice[symbol] - curPrice) == 0:
        return
    quant[symbol]['qa'] = quant[symbol]['usd'] / curPrice    # set Trading Amount in Quote Asset or Symbol
    quant[symbol]['qa'] = round(quant[symbol]['qa'] - quant[symbol]['qa'] % getStepSize(symbol), 8)    # set Trading Amount in Quote Asset or Symbol
    candles = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1DAY, "1 month ago UTC")
    temp = uptrend[symbol]
    if temp == -1:
        isFirstRun = True
    else:
        isFirstRun = False
    maClose = ma(True, 5, candles)       # EMAs
    maMiddle = ma(True, 8, candles, 0)
    maFar = ma(True, 13, candles, 0)
    dif = 100 * abs(maClose - maFar) / curPrice # Percentage difference in EMAs
    if maClose > maFar:
        uptrend[symbol]  = 1 
    elif maClose < maFar: 
        uptrend[symbol]  = 0
    if not isOrderOpen[symbol] and uptrend[symbol] > temp and not isFirstRun: 
        print("buy at "+s(curPrice*1.0005,6))
        print("quant qa:" + str(quant[symbol]['qa']))
        # order = client.order_market_buy(
        #    symbol= symbol,
        #    quantity=quant[symbol]['qa'])
        buys[symbol].append(curPrice*1.0005)
        print("Buy " + symbol + " at "+ '%s' % float('%.4g' % curPrice))
        if tweet("Buy " + symbol + " at "+ '%s' % float('%.4g' % curPrice)) < 0:
            symbols.remove(symbol)
            tweet('Removing '+symbol+' from trading pairs')
        
    if isOrderOpen[symbol] and maClose < maMiddle:
        print("sell at "+s(curPrice*0.9995,6) + " Profit: " + s(buys[symbol][-1]/curPrice,4))
        # order = client.order_market_sell(
        #    symbol= symbol,
        #    quantity=quant[symbol]['qa'])
        quant[symbol]['usd'] = quant[symbol]['qa'] * curPrice*0.9995   # Update USD value of Trading Amount
        sells[symbol].append(curPrice*0.9995)
        print("Sell " + symbol + " at " + '%s' % float('%.4g' % curPrice))
        if tweet("Sell " + symbol + " at " + '%s' % float('%.4g' % curPrice)) < 0:
            symbols.remove(symbol)
            tweet('Removing '+symbol+' from trading pairs')
        
    isOrderOpen[symbol] = len(buys[symbol]) - len(sells[symbol])
    if isOrderOpen[symbol]:
        dif = 100 * abs(maMiddle - maClose) / curPrice # % difference is order is open

    print(symbol + ': ' + '%s' % float('%.4g' % curPrice) + " d: " + s(dif) + "% up: " + s(uptrend[symbol]))
    lastPrice[symbol] = curPrice
    
print("Coin Price, MA % Dif, isUptrend")
# Start Trading
#looper()

Coin Price, MA % Dif, isUptrend


In [6]:
looper()

ETHBTC: 0.03162 d: 2.957% up: 0
LTCBTC: 0.008282 d: 2.663% up: 0
BNBBTC: 0.001547 d: 0.041% up: 0
NEOBTC: 0.002457 d: 4.477% up: 0
BCCBTC: 0.06983 d: 4.271% up: 0
GASBTC: 0.000784 d: 4.161% up: 0
MCOBTC: 0.000649 d: 2.028% up: 0
WTCBTC: 0.0004384 d: 1.151% up: 0
LRCBTC: 1.68e-05 d: 0.399% up: 1
QTUMBTC: 0.000551 d: 1.714% up: 0
YOYOBTC: 4.43e-06 d: 2.256% up: 0
OMGBTC: 0.000481 d: 3.459% up: 0
ZRXBTC: 0.0001114 d: 1.721% up: 1
STRATBTC: 0.0002197 d: 1.644% up: 0
SNGLSBTC: 4.23e-06 d: 3.895% up: 1
BQXBTC: 5.502e-05 d: 2.888% up: 1
KNCBTC: 6.346e-05 d: 1.994% up: 1
FUNBTC: 2.21e-06 d: 3.023% up: 0
SNMBTC: 8.18e-06 d: 1.186% up: 0
IOTABTC: 7.589e-05 d: 4.862% up: 0
LINKBTC: 5.156e-05 d: 0.354% up: 1
XVGBTC: 2.2e-06 d: 2.468% up: 0
SALTBTC: 8.54e-05 d: 0.156% up: 0
MDABTC: 0.0003276 d: 22.915% up: 1
MTLBTC: 0.0001092 d: 1.317% up: 1
SUBBTC: 1.642e-05 d: 4.052% up: 0
EOSBTC: 0.0008258 d: 1.863% up: 0
SNTBTC: 5.48e-06 d: 1.848% up: 0
ETCBTC: 0.001487 d: 4.244% up: 0
MTHBTC: 5.76e-06 d: 6.797

LTCBTC: 0.008286 d: 2.647% up: 0
BNBBTC: 0.001549 d: 0.008% up: 0
BCCBTC: 0.0699 d: 4.24% up: 0
MCOBTC: 0.00065 d: 1.979% up: 0
BQXBTC: 5.518e-05 d: 2.966% up: 1
KNCBTC: 6.326e-05 d: 1.907% up: 1
IOTABTC: 7.59e-05 d: 4.857% up: 0
MDABTC: 0.0003281 d: 22.918% up: 1
EOSBTC: 0.0008259 d: 1.859% up: 0
MTHBTC: 5.92e-06 d: 7.301% up: 1
ASTBTC: 1.335e-05 d: 0.091% up: 0
EVXBTC: 8.4e-05 d: 3.092% up: 1
REQBTC: 8.13e-06 d: 5.62% up: 1
XRPBTC: 6.832e-05 d: 4.031% up: 0
STORJBTC: 5.522e-05 d: 8.303% up: 1
ETHBTC: 0.03159 d: 2.98% up: 0
LTCBTC: 0.008289 d: 2.635% up: 0
BNBBTC: 0.001548 d: 0.016% up: 0
NEOBTC: 0.002456 d: 4.491% up: 0
BCCBTC: 0.06987 d: 4.253% up: 0
OMGBTC: 0.000482 d: 3.39% up: 0
BQXBTC: 5.537e-05 d: 3.058% up: 1
KNCBTC: 6.319e-05 d: 1.876% up: 1
IOTABTC: 7.589e-05 d: 4.862% up: 0
MDABTC: 0.0003285 d: 22.926% up: 1
EOSBTC: 0.0008256 d: 1.871% up: 0
ETCBTC: 0.001486 d: 4.267% up: 0
MTHBTC: 5.86e-06 d: 7.071% up: 1
ENGBTC: 9.212e-05 d: 0.933% up: 0
TRXBTC: 3.72e-06 d: 1.386% up: 1
X

BNBBTC: 0.001545 d: 0.086% up: 0
NEOBTC: 0.002458 d: 4.463% up: 0
ZRXBTC: 0.0001113 d: 1.686% up: 1
SNGLSBTC: 4.22e-06 d: 3.834% up: 1
FUNBTC: 2.21e-06 d: 3.039% up: 0
XVGBTC: 2.2e-06 d: 2.468% up: 0
EOSBTC: 0.0008255 d: 1.875% up: 0
MTHBTC: 5.92e-06 d: 7.301% up: 1
DNTBTC: 3.71e-06 d: 1.116% up: 0
OAXBTC: 3.535e-05 d: 2.219% up: 1
EVXBTC: 8.474e-05 d: 3.325% up: 1
VIBBTC: 6.44e-06 d: 0.43% up: 0
TRXBTC: 3.73e-06 d: 1.462% up: 1
KMDBTC: 0.0001816 d: 1.74% up: 1
LTCBTC: 0.008291 d: 2.627% up: 0
BNBBTC: 0.001546 d: 0.055% up: 0
NEOBTC: 0.002457 d: 4.465% up: 0
BCCBTC: 0.06985 d: 4.264% up: 0
FUNBTC: 2.22e-06 d: 2.892% up: 0
XVGBTC: 2.19e-06 d: 2.615% up: 0
MDABTC: 0.0003276 d: 22.907% up: 1
SUBBTC: 1.643e-05 d: 4.032% up: 0
EOSBTC: 0.0008269 d: 1.821% up: 0
ETCBTC: 0.001485 d: 4.29% up: 0
MTHBTC: 5.94e-06 d: 7.377% up: 1
ZECBTC: 0.01702 d: 4.408% up: 0
BTGBTC: 0.004001 d: 0.208% up: 1
EVXBTC: 8.447e-05 d: 3.24% up: 1
VIBBTC: 6.4e-06 d: 0.618% up: 0
TRXBTC: 3.72e-06 d: 1.386% up: 1
ETHBTC

TRXBTC: 3.72e-06 d: 1.395% up: 1
POWRBTC: 2.576e-05 d: 0.413% up: 1
MODBTC: 0.0001555 d: 11.578% up: 1
ETHBTC: 0.03156 d: 3.013% up: 0
BNBBTC: 0.001544 d: 0.093% up: 0
NEOBTC: 0.002457 d: 4.477% up: 0
BCCBTC: 0.06992 d: 4.23% up: 0
OMGBTC: 0.000484 d: 3.253% up: 0
ZRXBTC: 0.0001114 d: 1.706% up: 1
KNCBTC: 6.286e-05 d: 1.752% up: 1
IOTABTC: 7.592e-05 d: 4.848% up: 0
MDABTC: 0.0003292 d: 22.94% up: 1
MTLBTC: 0.0001103 d: 1.6% up: 1
EOSBTC: 0.0008248 d: 1.901% up: 0
DASHBTC: 0.02489 d: 3.975% up: 0
EVXBTC: 8.472e-05 d: 3.319% up: 1
XRPBTC: 6.826e-05 d: 4.052% up: 0
MODBTC: 0.0001553 d: 11.555% up: 1
ETHBTC: 0.03157 d: 3.007% up: 0
BCCBTC: 0.06995 d: 4.227% up: 0
BQXBTC: 5.517e-05 d: 2.957% up: 1
SALTBTC: 8.54e-05 d: 0.156% up: 0
MDABTC: 0.0003292 d: 22.94% up: 1
MTLBTC: 0.0001102 d: 1.574% up: 1
MTHBTC: 5.89e-06 d: 7.187% up: 1
TRXBTC: 3.71e-06 d: 1.319% up: 1
XRPBTC: 6.825e-05 d: 4.057% up: 0
MODBTC: 0.0001555 d: 11.578% up: 1
STORJBTC: 5.515e-05 d: 8.275% up: 1
ETHBTC: 0.03157 d: 3.01% 

ZRXBTC: 0.0001114 d: 1.703% up: 1
BQXBTC: 5.518e-05 d: 2.962% up: 1
IOTABTC: 7.587e-05 d: 4.871% up: 0
EOSBTC: 0.0008251 d: 1.89% up: 0
MTHBTC: 5.96e-06 d: 7.452% up: 1
ENGBTC: 9.172e-05 d: 1.067% up: 0
DASHBTC: 0.02485 d: 4.029% up: 0
EVXBTC: 8.46e-05 d: 3.281% up: 1
TRXBTC: 3.71e-06 d: 1.319% up: 1
XRPBTC: 6.823e-05 d: 4.067% up: 0
MODBTC: 0.000158 d: 11.865% up: 1
KMDBTC: 0.0001811 d: 1.663% up: 1
ETHBTC: 0.03156 d: 3.013% up: 0
LTCBTC: 0.008288 d: 2.639% up: 0
NEOBTC: 0.002454 d: 4.519% up: 0
BCCBTC: 0.0699 d: 4.242% up: 0
GASBTC: 0.000787 d: 4.031% up: 0
OMGBTC: 0.000484 d: 3.253% up: 0
ZRXBTC: 0.0001112 d: 1.653% up: 1
BQXBTC: 5.54e-05 d: 3.068% up: 1
MDABTC: 0.0003291 d: 22.937% up: 1
MTHBTC: 5.95e-06 d: 7.464% up: 1
ENGBTC: 9.18e-05 d: 1.04% up: 0
REQBTC: 8.19e-06 d: 5.797% up: 1
TRXBTC: 3.72e-06 d: 1.395% up: 1
XRPBTC: 6.824e-05 d: 4.062% up: 0
MODBTC: 0.0001575 d: 11.808% up: 1
ETHBTC: 0.03157 d: 3.003% up: 0
LTCBTC: 0.008289 d: 2.635% up: 0
BCCBTC: 0.06994 d: 4.221% up: 0
GA

...

DASHBTC: 0.02491 d: 3.947% up: 0
EVXBTC: 8.338e-05 d: 2.939% up: 1
TRXBTC: 3.72e-06 d: 1.395% up: 1
MODBTC: 0.0001492 d: 10.928% up: 1
ETHBTC: 0.03165 d: 2.956% up: 0
LTCBTC: 0.008304 d: 2.584% up: 0
BNBBTC: 0.001532 d: 0.286% up: 0
NEOBTC: 0.002453 d: 4.538% up: 0
BCCBTC: 0.07 d: 4.239% up: 0
GASBTC: 0.000781 d: 4.291% up: 0
QTUMBTC: 0.000553 d: 1.6% up: 0
OMGBTC: 0.000484 d: 3.275% up: 0
ZRXBTC: 0.0001117 d: 1.79% up: 1
BQXBTC: 5.524e-05 d: 2.99% up: 1
MDABTC: 0.0002784 d: 22.33% up: 1
MTLBTC: 0.0001084 d: 1.074% up: 1
ETCBTC: 0.001474 d: 4.524% up: 0
ENGBTC: 9.208e-05 d: 0.946% up: 0
DASHBTC: 0.02488 d: 3.994% up: 0
STORJBTC: 5.425e-05 d: 7.944% up: 1
KMDBTC: 0.0001819 d: 1.769% up: 1
ETHBTC: 0.03165 d: 2.957% up: 0
BNBBTC: 0.001532 d: 0.284% up: 0
NEOBTC: 0.002454 d: 4.524% up: 0
BCCBTC: 0.06994 d: 4.263% up: 0
WTCBTC: 0.0004402 d: 1.035% up: 0
QTUMBTC: 0.00055 d: 1.771% up: 0
YOYOBTC: 4.42e-06 d: 2.32% up: 0
OMGBTC: 0.000481 d: 3.481% up: 0
ZRXBTC: 0.0001119 d: 1.835% up: 1
BQXBTC

LTCBTC: 0.008315 d: 2.541% up: 0
BNBBTC: 0.001528 d: 0.349% up: 0
BCCBTC: 0.06986 d: 4.306% up: 0
WTCBTC: 0.0004427 d: 0.862% up: 0
ZRXBTC: 0.000112 d: 1.858% up: 1
MDABTC: 0.0002795 d: 22.311% up: 1
EOSBTC: 0.0008216 d: 2.017% up: 0
ETCBTC: 0.001473 d: 4.547% up: 0
DASHBTC: 0.02477 d: 4.126% up: 0
REQBTC: 7.93e-06 d: 5.103% up: 1
ARKBTC: 0.0001094 d: 0.1% up: 0
XRPBTC: 6.984e-05 d: 3.45% up: 0
STORJBTC: 5.409e-05 d: 7.879% up: 1
KMDBTC: 0.000182 d: 1.784% up: 1
ETHBTC: 0.03164 d: 3.001% up: 0
LTCBTC: 0.008313 d: 2.549% up: 0
BNBBTC: 0.001528 d: 0.353% up: 0
NEOBTC: 0.002453 d: 4.538% up: 0
BCCBTC: 0.06991 d: 4.279% up: 0
MCOBTC: 0.000646 d: 2.147% up: 0
OMGBTC: 0.000482 d: 3.412% up: 0
ZRXBTC: 0.0001121 d: 1.885% up: 1
STRATBTC: 0.0002209 d: 1.474% up: 0
IOTABTC: 7.597e-05 d: 4.826% up: 0
MDABTC: 0.000278 d: 22.27% up: 1
EOSBTC: 0.0008221 d: 1.998% up: 0
ZECBTC: 0.01703 d: 4.415% up: 0
EVXBTC: 8.336e-05 d: 2.951% up: 1
TRXBTC: 3.71e-06 d: 1.319% up: 1
ARKBTC: 0.0001092 d: 0.128% up: 0

KNCBTC: 6.281e-05 d: 1.732% up: 1
FUNBTC: 2.19e-06 d: 3.355% up: 0
LINKBTC: 5.164e-05 d: 0.399% up: 1
MDABTC: 0.0002808 d: 22.344% up: 1
MTLBTC: 0.000109 d: 1.231% up: 1
EOSBTC: 0.0008207 d: 2.045% up: 0
MTHBTC: 5.8e-06 d: 6.75% up: 1
DASHBTC: 0.02486 d: 4.009% up: 0
EVXBTC: 8.335e-05 d: 2.948% up: 1
XRPBTC: 6.949e-05 d: 3.596% up: 0
MODBTC: 0.0001485 d: 10.839% up: 1
ENJBTC: 7.4e-06 d: 3.293% up: 0
ETHBTC: 0.03162 d: 2.99% up: 0
LTCBTC: 0.008298 d: 2.607% up: 0
BCCBTC: 0.06988 d: 4.294% up: 0
LRCBTC: 1.657e-05 d: 0.009% up: 0
IOTABTC: 7.555e-05 d: 5.018% up: 0
LINKBTC: 5.17e-05 d: 0.433% up: 1
MDABTC: 0.0002809 d: 22.345% up: 1
MTLBTC: 0.0001087 d: 1.153% up: 1
EOSBTC: 0.0008209 d: 2.044% up: 0
DASHBTC: 0.02479 d: 4.101% up: 0
EVXBTC: 8.334e-05 d: 2.945% up: 1
TRXBTC: 3.7e-06 d: 1.252% up: 1
XRPBTC: 6.95e-05 d: 3.613% up: 0
MODBTC: 0.0001484 d: 10.826% up: 1
STORJBTC: 5.451e-05 d: 8.047% up: 1
ETHBTC: 0.03162 d: 2.989% up: 0
BCCBTC: 0.06986 d: 4.307% up: 0
LRCBTC: 1.655e-05 d: 0.044% 

sell at 1.7e-05 Profit: 1.0059
Sell LRCBTC at 1.654e-05
Sell LRCBTC at 1.654e-05 -Bot 2018-10-15 23:42:34.986236
LRCBTC: 1.654e-05 d: 0.062% up: 0
OMGBTC: 0.000482 d: 3.412% up: 0
STRATBTC: 0.0002205 d: 1.53% up: 0
FUNBTC: 2.19e-06 d: 3.339% up: 0
IOTABTC: 7.555e-05 d: 5.018% up: 0
MDABTC: 0.0002901 d: 22.657% up: 1
SUBBTC: 1.651e-05 d: 3.866% up: 0
EOSBTC: 0.000821 d: 2.04% up: 0
ETCBTC: 0.001473 d: 4.547% up: 0
MTHBTC: 5.76e-06 d: 6.59% up: 1
ZECBTC: 0.01696 d: 4.545% up: 0
OAXBTC: 3.597e-05 d: 2.638% up: 1
XRPBTC: 6.948e-05 d: 3.622% up: 0
MODBTC: 0.0001486 d: 10.852% up: 1
STORJBTC: 5.454e-05 d: 8.059% up: 1
ETHBTC: 0.03163 d: 2.975% up: 0
LTCBTC: 0.008298 d: 2.607% up: 0
BNBBTC: 0.001531 d: 0.292% up: 0
NEOBTC: 0.002451 d: 4.566% up: 0
MCOBTC: 0.000646 d: 2.147% up: 0
WTCBTC: 0.0004374 d: 1.232% up: 0
buy at 1.7e-05
quant qa:601443.0
Buy LRCBTC at 1.661e-05
Buy LRCBTC at 1.661e-05 -Bot 2018-10-15 23:42:53.473584
LRCBTC: 1.661e-05 d: 0.397% up: 1
ZRXBTC: 0.0001118 d: 1.773% up: 1
B

Exception in thread Thread-651:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 387, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py", line 383, in _make_request
    httplib_response = conn.getresponse()
  File "/anaconda3/lib/python3.6/http/client.py", line 1331, in getresponse
    response.begin()
  File "/anaconda3/lib/python3.6/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/anaconda3/lib/python3.6/http/client.py", line 258, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/anaconda3/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
  File "/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py", line 294, in recv_into
    raise timeout('The read operation timed out')
socket.timeo

In [11]:
str(datetime.now())

'2018-10-15 20:44:14.191360'

In [49]:
tweet('Alright the bot is working. I gave it around 60 trading pairs (ETHBTC, XRPBTC, etc) for testing and removed its ability to buy or sell.')

BTCUSDT: 6738.0 d: 0.317% up: 0
BTCUSDT: 6737.0 d: 0.32% up: 0
ETHUSDT: 213.9 d: 3.229% up: 0
BTCUSDT: 6740.0 d: 0.31% up: 0
BTCUSDT: 6739.0 d: 0.313% up: 0
BTCUSDT: 6740.0 d: 0.31% up: 0
ETHUSDT: 213.9 d: 3.228% up: 0
BTCUSDT: 6739.0 d: 0.313% up: 0
ETHUSDT: 213.9 d: 3.217% up: 0
ETHUSDT: 213.9 d: 3.228% up: 0
ETHUSDT: 213.9 d: 3.226% up: 0
BTCUSDT: 6741.0 d: 0.304% up: 0
ETHUSDT: 213.9 d: 3.217% up: 0
ETHUSDT: 214.0 d: 3.211% up: 0
ETHUSDT: 213.9 d: 3.228% up: 0
BTCUSDT: 6739.0 d: 0.313% up: 0
ETHUSDT: 213.8 d: 3.238% up: 0
BTCUSDT: 6740.0 d: 0.309% up: 0
BTCUSDT: 6739.0 d: 0.313% up: 0
BTCUSDT: 6740.0 d: 0.309% up: 0
BTCUSDT: 6743.0 d: 0.304% up: 0
BTCUSDT: 6740.0 d: 0.308% up: 0
ETHUSDT: 213.8 d: 3.241% up: 0
BTCUSDT: 6743.0 d: 0.297% up: 0
ETHUSDT: 213.9 d: 3.226% up: 0
BTCUSDT: 6741.0 d: 0.308% up: 0
ETHUSDT: 213.9 d: 3.224% up: 0
ETHUSDT: 213.8 d: 3.237% up: 0
BTCUSDT: 6740.0 d: 0.308% up: 0
ETHUSDT: 213.7 d: 3.248% up: 0
BTCUSDT: 6741.0 d: 0.307% up: 0
ETHUSDT: 213.7 d: 3.257% 